<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/5)_Consigna_36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***La 3-upla de palabras más común en los comentarios de los logs (2)***

Realizo la installacion de las librerias que utilizare.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!pip install --upgrade pandas
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!pip install nltk==3.4

In [ ]:
!pip install pyarrow

Importo librerias.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pyarrow

Creo el sparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo al google drive, para acceder al archivo.

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Leo el archivo.

In [ ]:
logs = sqlContext.read.parquet('gdrive/My Drive/TP2 - Datos/logs.parquet')

In [ ]:
rddLogs = logs.rdd

In [ ]:
rddLogs.take(2)

[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel')]

Creo la funcion de obtener ngramas.

In [ ]:
symbols = ['!',',','"','#','$','%','&','\',''','(',')','()','*','+','-','.','/',':',';','<','=','>','?','@','[','\,',']','^','_','`','``','','{','|','}','{}','[]','~','pipe', "''", "'", '||', '==','--']

In [ ]:
def obtener_ngrams(texto, n):
    text = texto.lower()
    text = word_tokenize(text)
    list_text = []

    for w in text:
        if w not in symbols:    #quito los simbolos sueltos
            list_text.append(w) 
    
    n_grams = ngrams(list_text, n)
    return n_grams

Utilizare un 1% de los comments del logs, por cuestiones de memoria.

In [ ]:
rddComment = rddLogs.sample(False,0.01)

Limpio el rdd que voy a utilizar.

In [ ]:
rddComment = rddComment.filter(lambda x: x.comment != None)

Aplico un flatMap para aplicar la funcion de los ngramas sobre los comentarios.

In [ ]:
rddComment = rddComment.flatMap(lambda x: [y for y in obtener_ngrams(x.comment,3)]) #obtener_ngrams recibe el texto y el n para los n-grams

In [ ]:
rddComment.take(2)

[('el', 'contenido', 'era'), ('contenido', 'era', "'si")]

Realizo un map para tener la estructura --> ((ngrama),1) y luego un reduceByKey para obtener la frecuencia de ese ngrama.

 ***Resultado***

In [ ]:
rddComment.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).takeOrdered(1,lambda x: -x[1])

[(('el', 'contenido', 'era'), 11398)]